In [ ]:
print('Setup complete.')

# Lab 01: Introduction to MCPS (Model-centric Protocol Suite)

## Learning Objectives
- Understand what MCPS is and why it's important for building advanced agents
- Learn about the core components: MCP Servers, Resources, and Tools
- Implement a mock MCP Server and register a simple tool
- Write code to list and interact with resources from the server

## Setup

In [ ]:
from typing import List, Dict, Any, Callable
from dataclasses import dataclass, field

## Part 1: Core Concepts of MCPS

MCPS is a standardized way for AI models (like LLMs) to interact with the outside world. It provides a structured protocol for discovering and using external tools and services.

- **MCP Server**: A service that exposes a set of capabilities. For example, a `git` server could expose tools for cloning repos and committing files.
- **Resource**: A specific capability or function exposed by a server. This is often called a 'tool' in other contexts (like OpenAI's function calling).
- **Tool Call**: The action of an AI model requesting to use a specific resource with certain arguments.

In [ ]:
# --- Mock Tool Implementation ---
def get_server_time() -> str:
    """A simple tool that returns the current server time."""
    from datetime import datetime
    return datetime.now().isoformat()

# --- Mock MCP Server ---
@dataclass
class MCPResource:
    name: str
    description: str
    handler: Callable[[], Any]

class MockMCPServer:
    def __init__(self, name: str):
        self.name = name
        self.resources: Dict[str, MCPResource] = {}

    def register_resource(self, resource: MCPResource):
        """Makes a tool available through the server."""
        self.resources[resource.name] = resource
        print(f'Registered resource "{resource.name}" on server "{self.name}".')

    def list_resources(self) -> List[Dict[str, str]]:
        """Lists the available tools and their descriptions."""
        return [{'name': r.name, 'description': r.description} for r in self.resources.values()]

    def execute_resource(self, name: str, args: Dict = None) -> Any:
        """Executes a tool by its name."""
        if name not in self.resources:
            return f'Error: Resource "{name}" not found.'
        
        handler = self.resources[name].handler
        try:
            return handler(**(args or {}))
        except Exception as e:
            return f'Error executing resource "{name}": {e}'

## Part 2: Setting Up and Interacting with a Server

In [ ]:
# 1. Initialize the server
system_tools_server = MockMCPServer(name='system-tools')

# 2. Create and register a resource
time_resource = MCPResource(
    name='get_server_time',
    description='Returns the current ISO-formatted server time.',
    handler=get_server_time
)
system_tools_server.register_resource(time_resource)

# 3. An agent discovers the available tools
print("--- Agent discovers available resources ---")
available_resources = system_tools_server.list_resources()
print(available_resources)

# 4. An agent decides to use a tool and executes it
print("--- Agent executes a resource ---")
# This would typically be triggered by an LLM's function call output
tool_to_call = 'get_server_time'
result = system_tools_server.execute_resource(tool_to_call)
print(f'Result of calling "{tool_to_call}": {result}')

## Part 3: A Mock Agent using the MCPS Server

In [ ]:
class MCPSAgent:
    def __init__(self, server: MockMCPServer):
        self.server = server

    def process_query(self, query: str) -> str:
        # Mock LLM logic: if the query asks for the time, use the tool.
        if 'time' in query.lower():
            print('Agent decided to use the get_server_time tool.')
            # In a real system, this would be a structured tool call
            tool_name = 'get_server_time'
            
            # Execute and get the observation
            observation = self.server.execute_resource(tool_name)
            
            # Formulate a final response
            return f'The current server time is {observation}.'
        return "I can't answer that, but I can get the server time."

# --- Run the agent ---
agent = MCPSAgent(server=system_tools_server)
user_query = 'What time is it on the server?'
response = agent.process_query(user_query)

print(f'\nUser Query: {user_query}')
print(f'Agent Response: {response}')

## Exercises

1. **Register a New Resource**: Create a new function called `get_server_name()` that returns the server's name. Create a new `MCPResource` for it and register it with the `system_tools_server`. Then, modify the `MCPSAgent` to handle a query like, "What is the server's name?".
2. **Add a Resource with Arguments**: Create a new function `echo(message: str) -> str` that simply returns the message it was given. Register it as a resource. You will need to modify `MCPResource` and `execute_resource` to handle arguments. Finally, update the agent to use this tool for a query like, "Echo this message: Hello, world!".
3. **Create a Second Server**: Instantiate a second `MockMCPServer` called `file-system-server`. Create and register a `list_files(path: str)` tool on it. This demonstrates how an agent could connect to multiple MCP servers to access different domains of functionality.

## Summary

You learned:
- **MCPS** provides a standardized interface between AI models and external tools.
- An **MCP Server** acts as a gateway, exposing a collection of **Resources** (tools).
- An agent can **discover** what tools are available by listing resources and then **execute** them to perform actions or gather information.
- This architecture decouples the AI's reasoning from the tool's implementation, making the system more modular and scalable.